This notebook will be an exploration of cold start methodologies in recomender systems.  I will use the Yelp dataset.

In [15]:
import pandas as pd
import sqlalchemy
import os
import tarfile
import json

# Import json and add to database

In [4]:
os.listdir('data')

['yelp_dataset.tar', 'yelp_dataset']

In [6]:
if not os.path.exists('data/yelp_dataset'):
    tar = tarfile.open(os.path.join('data', 'yelp_dataset.tar'))
    tar.extractall('data/yelp_dataset')
else:
    print("Destination already exists.")

Destination already exists.


In [7]:
DATA_PATH = 'data/yelp_dataset'
os.listdir(DATA_PATH)

['yelp_academic_dataset_checkin.json',
 'Dataset_User_Agreement.pdf',
 'yelp_academic_dataset_tip.json',
 'yelp_academic_dataset_review.json',
 'yelp_academic_dataset_business.json',
 'yelp_academic_dataset_user.json']

## Reviews
Reviews is too large to hold in memory, we need to write this directly to a database line by line.

In [23]:
review_file = os.path.join(DATA_PATH, 'yelp_academic_dataset_review.json')

with open(review_file, 'r') as f:
    line = f.readline()
    line = json.loads(line)
    line = {k: [v] for k, v in line.items()}
    line_df = pd.DataFrame.from_dict(line)
    
    line_df.to_sql
line_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   review_id    1 non-null      object 
 1   user_id      1 non-null      object 
 2   business_id  1 non-null      object 
 3   stars        1 non-null      float64
 4   useful       1 non-null      int64  
 5   funny        1 non-null      int64  
 6   cool         1 non-null      int64  
 7   text         1 non-null      object 
 8   date         1 non-null      object 
dtypes: float64(1), int64(3), object(5)
memory usage: 200.0+ bytes


In [ ]:
# file too large, we run out of memory
# review_df = pd.read_json(os.path.join(DATA_PATH, 'yelp_academic_dataset_review.json'), lines=True)

## Business

In [15]:
business_df = pd.read_json(os.path.join(DATA_PATH, 'yelp_academic_dataset_business.json'), lines=True)

In [16]:
business_df.shape

(160585, 14)

In [17]:
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",None
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'..."


In [19]:
business_df.categories.str.contains('Italian').sum()

3580